In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 파일 경로
file_paths = {
    "가임 여성수": "가임 여성수_full.csv",
    "사교육비 총액": "사교육비 총액_full.csv",
    "집값(평균)": "집값(평균)_full.csv",
    "혼인건수": "혼인건수_full.csv"
}

# 데이터 불러오기
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# 스케일링 작업
scaler = StandardScaler()
scaled_dataframes = {}

In [ ]:
# 데이터를 정리하여 숫자 형식으로 변환
for name, df in dataframes.items():
    # 모든 열에 대해 쉼표 제거 및 숫자로 변환 시도
    for col in df.columns[1:]:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
    dataframes[name] = df

# 다시 스케일링 작업 수행
scaled_dataframes = {}
for name, df in dataframes.items():
    scaled_df = pd.DataFrame(scaler.fit_transform(df.iloc[:, 1:]), columns=df.columns[1:])
    scaled_df.insert(0, df.columns[0], df.iloc[:, 0])  # 첫 번째 열(예: 지역) 유지
    scaled_dataframes[name] = scaled_df

# 스케일링된 데이터 저장 및 확인
for name, scaled_df in scaled_dataframes.items():
    output_path = f"{name}_scaled.csv"
    scaled_df.to_csv(output_path, index=False)

scaled_dataframes

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 종속 변수 데이터 로드
birth_data = pd.read_csv("시군구_출생아수_full.csv")

# 데이터 정리
# 독립 변수 및 종속 변수 병합
merged_data = birth_data.copy()
for name, scaled_df in scaled_dataframes.items():
    # suffixes 변경하여 중복 열 이름 방지
    merged_data = merged_data.merge(scaled_df, on=scaled_df.columns[0], how="inner", suffixes=('', f'_{name}'))

# 독립 변수(X)와 종속 변수(y) 분리
X = merged_data.iloc[:, 2:]  # 독립 변수
y = merged_data.iloc[:, 1]   # 종속 변수 (출생아 수)

# 결측치 제거
X = X.dropna()
y = y[X.index]

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 라쏘 회귀 모델 훈련
lasso_model = LassoCV(cv=5, random_state=42)
lasso_model.fit(X_train, y_train)

# 테스트 데이터 예측
y_pred = lasso_model.predict(X_test)

# 성능 평가
mse = mean_squared_error(y_test, y_pred)
selected_features = np.sum(lasso_model.coef_ != 0)  # 선택된 특징 개수

{
    "MSE": mse,
    "Selected Features": selected_features,
    "Lasso Coefficients": lasso_model.coef_,
    "Intercept": lasso_model.intercept_
}


In [ ]:
# 정확도 (R^2 스코어) 계산
r2_score = lasso_model.score(X_test, y_test)
r2_score